## Partie exrtaction des données

In [1]:
# import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random as rd
import xlrd

ModuleNotFoundError: No module named 'xlrd'

In [ ]:
# import bibliothèque traitement des fichiers tableur + ouverture du document
doc = xlrd.open_workbook('forestfires.csv')

In [ ]:
# extraction première (et seule) feuille du fichier + ses dimensions
feuille = doc.sheet_by_index(
lignes, colonnes = feuille.nrows, feuille.ncols

In [ ]:
# extraction noms des caractéristiques
feature_names = []
for i in range(colonnes - 1): # -1 pour enlever la dernière colonne (résuultats)
    # on prend pas les coordonnées (colonnes 0 et 1)
    if i >= 2:
        feature_names.append(feuille.cell_value(rowx = 0, colx = i))

In [ ]:
# convertir les mois et les jours de chaine en entiers
month = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
day = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']

def month_2_int(i):
    n = len(month)
    for k in range(n):
        if feuille.cell_value(rowx = i, colox = 2) == month[k]:
            return k

def day_2_int(i):
    n = len(day)
    for k in range(n):
        if feuille.cell_value(rowx = i, colox = 3) == day[k]:
            return k

In [ ]:
# extraction données dans tableau numpy (scikit avec type Bunch)
data_list = []
for i in range(1, lignes):
    elem = []
    for j in range(colonnes - 1):
        # omission volontaire des colonnes de coordonnées 0 et 1
        if j>= 4:
            elem.append(float(feuille.cell_value(rowx = i, colx = j)))
        elif j == 2: # mois transformés
            elem.append(month_2_int(i))
        elif j == 3: # jours transformés
            elem.append(day_2_int(i))
    data_list.append(elem)
    elem = []

data = np.array(data_list)

In [ ]:
# mettre les targets dans un tableau numpy de 1 colonne target
target_temp = []
for i in range(1, lignes):
    target_temp.append(float(feuille.cell_value(rowx=i, colx=12)))

target = np.array([np.log(1+X) for X in target_temp]) # transfo logarithmique
# rapprocher valeurs des abscisses (sans ça: imprécisions pour grandes valeurs)

fire= [data, feature_names, target]

In [ ]:
# fabrication d'un second set pour un second modèle sur de petits incendies
# calcul proportion d'échantillons de target < = 15 hectares
n = len(target)
s = 0
for i in range(n):
    if target_temp[i] <= 15:
        s += 1

# qui représentent +85% des échantillons
print(s/n)

In [ ]:
dat = []
targ = []
for i in range(n):
    if target_temp[i] < 15:
        targ.append(target[i])
        dat.append(data[i])
targ = np.array([np.log(1+X) for X in targ])
dat = np.array(dat)

# Sur la nécessité d'une transformation logarithmique

In [ ]:
serie = pd.Series(target_temp).plot.hist(grid=True, bins=40, rwidth=.9, color="#000000")
plt.title('Allure de la distribution des surfaces')
plt.xlabel('Surface brûlée')
plt.ylabel('Fréquence')
plt.grid(axis='y', alpha=0.75)

In [ ]:
serie = pd.Series(target).plot.hist(grid=True, bins=40, rwidth=.9, color="#000000")
plt.title('Allure de la distribution des surfaces')
plt.xlabel('Surface brûlée')
plt.ylabel('Fréquence')
plt.grid(axis='y', alpha=0.75)

# Partie division du dataset

In [ ]:
# dans le module model_selection, train_test_split pour diviser les data
from sklearn.model_selection import train_test_split

# séparation en 2 jeux (un de train et un de test)
data_split = train_test_split(data, target, train_size=0.75, test_size =0.25, random_state =70, shuffle=False)
data_train, data_test, target_train, target_test = data_split

# une seconde fois 2 jeux (cette fois-ci target <= 5 hectares uniquement)
dat_spl = train_test_split(dat, targ, train_size =.75, test_size=.25, random_state=70, shuffle=False)
dat_tra, dat_tst, targ_tra, targ_tst = dat_spl

# Partie mise à l'échelle des données

In [2]:
from sklearn.preprocessing import StandardScaler

scalerS = StandardScaler() # pour toutes les données
scalerS2 = StandardScaler() # les données ayant target <= 15 ha

scalerS.fit(data_train), scalerS2.fit(dat_tra)

data_train_tr = scalerS.transform(data_train)
data_test_tr = scalerS.transform(data_test)
dat_tra_tr, dat_tst_tr = scalerS2.transform(dat_tra), scalerS2.transform(dat_tst)

NameError: name 'data_train' is not defined

# Partie calcul du score

In [3]:
def score(vrais_resultats, predictions, eps=10):
    # renvoie RMSE et % de points tels que abs(prediction-résultat) <= eps
    ecart = np.abs(vrais_resultats - predictions).ravel()
    # ravel pour aplatir un éventuel tableau de plusieurs dimensions
    RMSE, s, n = 0, 0, len(ecart)
    for i in range(n):
        RMSE += ecart[i]*ecart[i]
        if ecart[i] <= eps:
            s += 1
    return np.sqrt(RMSE/n), 100*(round(100*s/n)/100)

# Méthode de recherche heuristique de C (ici pour le modèle 2)

In [ ]:
from sklearn.svm import SVR
eps = 1e-1

Y3, Y4 = [], []
CC = [i/10 for i in range(1, 10)] + [i for i in range(1, 11)]

for C in CC:
    model2 = SVR(C=C, gamma='auto', kernel='rbf', epsilon=eps)
    model2.fit(dat_tra_tr, targ_tra)
    log_predictions2 = model2.predict(dat_tst_tr)
    predictions2 = [np.exp(X)-1 for X in log_predictions2]
    rms, per = score(targ_tst, predictions2, eps=1)
    Y3.append(round(100*rms)/100), Y4.append(per)

In [ ]:
rms_graph = plt.scatter(CC,Y3, color='blue', marker='o')
plt.grid()
plt.xlabel("C")
plt.ylabel("RMSE")
plt.title("Comparaison de l'erreur moyenne quadratique du modèle 2 en fonction de C")
plt.legend([rms_graph], ['rms'], loc='lower right')
plt.show()

In [ ]:
per_graph = plt.scatter(CC,Y4, color='orange')
plt.grid()
plt.xlabel("C")
plt.ylabel("efficacité à 1 hectare près")
plt.title("Comparaison de l'efficacité du modèle 2 en fonction de C")
plt.legend([per_graph], ['per ha'], loc='lower right')
plt.show()

# Partie initialisation de l'algorithme

In [ ]:
from sklearn.svm import SVR
C = 0.2
eps = 1e-1

model = SVR(C=C, gamma='auto', kernel='rbf', epsilon= eps)
model.fit(data_train_tr, target_train)
model2 = SVR(C=C, gamma='auto', kernel='rbf', epsilon=eps)
model2.fit(dat_tra_tr, targ_tra)

In [ ]:
# ce qu'on peut retrouver à propos du modèle (ici model) après l'apprentissage
# attributs appelables: support_ , support_vectors_ , dual_coef_ , intercept_

n_sv = len(model.support_)
k = rd.randint(0, n_sv - 1)

In [ ]:
print('- Le {}ème vecteur de support est le vecteur numéro {} du jeu de départ.'.format(k, model.support_[k]))

sv_k = model.support_vectors_[k]
print("- Les {} coordonnées de ce vecteur dans l'espace de départ sont \n {} ".format(len(sv_k), sv_k))

print('- Son coefficient dual est {}'.format(model.dual_coef_[0, k]))

# Partie prédiction des modèles (tests)

In [ ]:
# prédictions model (encore sous forme logarithmique)
log_predictions = model.predict(data_test_tr)
predictions = [np.exp(X)-1 for X in log_predictions]

# prédictions model2 (encore sous forme logarithmique)
log_predictions2 = model2.predict(dat_tst_tr)
predictions2 = [np.exp(X)-1 for X in log_predictions2]

# Représentation logarithmique

In [ ]:
X, Y1, Y2 = [], [], []
for i in range(1,8):
    Y1.append(score(target_test, predictions, eps=i)[1])
    Y2.append(score(targ_tst, predictions2, eps=i)[1])
    X.append(i)
    

mod1 = plt.scatter(X,Y1, color='red', marker='o')
mod2 = plt.scatter(X,Y2, color='green', marker='*')
plt.grid()
plt.xlabel("hectares d'erreur acceptable")
plt.ylabel("efficacité en %")
plt.title("Comparaison de l'efficacité des 2 modèles")
plt.legend([mod1,mod2], ['modèle complet', 'modèle 15 ha'], loc='lower right')

In [ ]:
from ipywidgets import interact

@interact(n=(1,5))
def eps_change(n=2):
    global rms, per
    rms, per= score(target_test, predictions, eps=n)
    rms = round(abs(rms)*1e5)/1e5
    print('Toutes les données: {} (RMSE), {}% (pourcentage de réussite)'.format(rms, per))
    rms, per = score(targ_tst, predictions2, eps=n)
    rms = round(abs(rms)*1e5)/1e5
    print('Les données pour target < 15 ha: {} (RMSE), {}% (pourcentage de réussite)'.format(rms, per))


# Partie visualisation de la précision des prédictions

In [ ]:
def visu_diff_abs(pred, target):
    # visualiser la différence absolue entre deux tableaux
    tst = len(target)
    return [round(abs(target[k]-pred[k])*100)/100 for k in range(tst)]

def visu_diff_rel(pred, target):
    # visualiser la différence relative entre 2 tableaux
    tst = len(target)
    L = []
    for i in range(tst):
        if target[i] == 0:
            L.append((-1)*round(abs(target[i]-pred[i])*100)/100)
            continue
        L.append(round(abs((target[i]-pred[i])/target[i])*100)/100)
    
    return L

diff_abs = visu_diff_abs(predictions, target_test)
diff_abs2 = visu_diff_abs(predictions2, targ_tst)
diff_rel = visu_diff_rel(predictions, target_test)
diff_rel2 = visu_diff_rel(predictions2, targ_tst)


In [ ]:
print(sorted(diff_abs))

In [ ]:
print(sorted(diff_abs2))

In [ ]:
print(sorted(diff_rel))

In [ ]:
print(sorted(diff_rel2))

In [ ]:
# calcul R^2 second modèle (sur les données d'entraînement)

p = [np.exp(X)-1 for X in model2.predict(dat_tra_tr)]
n = len(model2.support_) # nb de SV
u, v = 0, 0
for i in range(n):
    v += (targ_tra[i]-p[i])**2
    if not i in model2.support_:
        u += (targ_tra[i]-p[i])**2

print(1 - u/v)

In [ ]:
# même calcul pour le premier modèle

p = [np.exp(X)-1 for X in model.predict(dat_tra_tr)]
n = len(model.support_) # nb de SV
u, v = 0, 0
for i in range(n):
    v += (targ_tra[i]-p[i])**2
    if not i in model.support_:
        u += (targ_tra[i]-p[i])**2

print(1 - u/v)